# Arbitrary Scoring

In [1]:
import numpy as np
import pandas as pd
import os, glob, dill
from collections import defaultdict, Counter
%matplotlib inline
import matplotlib.pyplot as plt
#import seaborn as sns

I built two different objects for each game. The first includes player information and stores things as DataFrames, the other minGame, does not and is lighter weight. Each has a method to rescore games.

In [165]:
class game():
    def __init__(self, df):
        '''
        Initializes the object pulling out the shots distances for each team as well as free throws and score at
        the end of regulation
        '''
        df=df[df['period']<=4]
        self.teams = list(df.team.dropna().unique())
        self.regscoring = dict(df[(df['shot_distance'].fillna(-1) >= 0) & (df['result'] == 'made')]\
                               .groupby('team')['points'].sum())
        self.shots = dict(list(df[(df['shot_distance'].fillna(-1) >= 0) & (df['result'] == 'made')]\
                               [['team','player','shot_distance','points']]\
                               .groupby('team')[['player','shot_distance','points']]))
        self.allShots = dict(list(df[(df['shot_distance'].fillna(-1) >= 0) ]\
                               [['team','player','shot_distance','points']]\
                               .groupby('team')[['player','shot_distance','points']]))
        self.fts = dict(list( df[(df['event_type'] == 'free throw') & (df['result'] == 'made')]\
                             [['team','player']].dropna().groupby(['team'])))
        self.reScores = {}
        self.regScores = {}
        self.pScores = []
        
    def newScores(self, func, fts = True):
        '''
        Rescores eqach game by applying some function to each shot 
        '''
        for team in self.teams:
            self.shots[team]['new'] = self.shots[team]['shot_distance'].apply(lambda x: func(x))
            self.reScores[team] = sum(self.shots[team]['new'])
            self.regScores[team] = self.regscoring[team]
            if fts:
                ft = self.fts[team].count()['team']
                self.reScores[team] += ft
                self.regScores[team] += ft
    
    def emitData(self):
        '''
        Decides who wins the game and emits the score both under the new and old scoring system
        '''
        scr = self.reScores.items()
        return [(scr[0][0], 1.* (scr[0][1]-scr[1][1])/max([1, scr[0][1], scr[1][1]]),\
                 scr[0][1], scr[1][1], self.regScores[scr[0][0]],  self.regScores[scr[1][0]])\
                ,(scr[1][0], 1.* (scr[1][1]-scr[0][1])/max([1, scr[0][1],scr[1][1]]),\
                  scr[1][1], scr[0][1], self.regScores[scr[1][0]],  self.regScores[scr[0][0]])]#\
                #, self.pScores
    
    def playerScoring(self, fts = True):
        '''
        Calculates scoring for each player
        '''
        self.pScores = []
        for team in self.teams:
            pts = Counter(dict(self.shots[team].groupby('player')['new'].sum()))
            if fts:
                pts += Counter(dict(self.fts[team].groupby('player').count()['team']))
            self.pScores.extend(pts.items())
    
    def playGame(self, func, fts = True):
        self.newScores(func, fts)
        #self.playerScoring(fts)
        return self.emitData()
                

In [140]:
class minGame():
    '''
    A lighter weight version of the game object which doesn't use DataFrames.
    '''
    def __init__(self, df):
        df=df[df['period']<=4]
        self.teams = list(df.team.dropna().unique())
        self.regscoring = dict(df[(df['shot_distance'].fillna(-1) >= 0) & (df['result'] == 'made')]\
                               .groupby('team')['points'].sum())
        self.shots = dict(map(lambda (i,j): (i, {'shot_distance' : j.as_matrix()[:,1],\
                       'points': j.as_matrix()[:,2]
                      }) , list(df[(df['shot_distance'].fillna(-1) >= 0) & (df['result'] == 'made')]\
                               [['team','shot_distance','points']]\
                               .groupby('team')[['shot_distance','points']])))
        self.fts = dict(df[(df['event_type'] == 'free throw') & (df['result'] == 'made')]\
                             [['team','player']].dropna().groupby(['team']).count()['player'])
        self.reScores = {}
        self.regScores = {}
        self.pScores = []
        
    def newScores(self, func, fts = True):
        for team in self.teams: 
            self.reScores[team] = sum(map(lambda x: func(x), self.shots[team]['shot_distance']))
            self.regScores[team] = self.regscoring[team]
            if fts:
                ft = self.fts[team]
                self.reScores[team] += ft
                self.regScores[team] += ft
    
    def emitData(self):
        scr = self.reScores.items()
        return [(scr[0][0], 1.* (scr[0][1]-scr[1][1])/max([1, scr[0][1], scr[1][1]]),\
                 scr[0][1], scr[1][1], self.regScores[scr[0][0]],  self.regScores[scr[1][0]])\
                ,(scr[1][0], 1.* (scr[1][1]-scr[0][1])/max([1, scr[0][1],scr[1][1]]),\
                  scr[1][1], scr[0][1], self.regScores[scr[1][0]],  self.regScores[scr[0][0]])]#\
                #, self.pScores
    
    def playGame(self, func, fts = True):
        self.newScores(func, fts)
        return self.emitData()
                

We also save all the games

In [168]:
games = []

for csv in glob.glob('2015-16/*.csv'):
    g1 = pd.read_csv('{}'.format(csv))
    if g1['data_set'].unique()[0] != '2015-2016 Regular Season':
        continue
    else:
        try:
            games.append(minGame(g1))

        except:
            print csv
            
dill.dump(games,open('ArbitraryScoringApp/gameData','wb'))

From games, we can collect how each player did in that game and make a list of qualified players and their shot distances per game

In [157]:
def collectPlayers(game):
    '''
    For a single game returns a dictionary of how all the players (that scored) did in that game
    '''
    ww = {}
    for team in game.teams:
        w = dict(list(game.shots[team][['player','shot_distance','points']].groupby('player')))
        z = dict(game.fts[team].groupby('player').count()['team'])
        v = dict(list(game.allShots[team][['player','shot_distance','points']].groupby('player')))
        for i in set(w.keys() + z.keys()):
            try:
                a = list(w[i]['shot_distance'])
                b = list(w[i]['points'])
            except:
                a, b = [], []
            try:
                c = z[i]
            except:
                c = 0
            try:
                d = v[i].count()['player']
            except:
                d=0
            ww[i] = {'shot_distance' : a, 'points' : b, 'fts': c, 'shots': d, 'games': 1 } 
            
    return ww

In [174]:
games = []

for csv in glob.glob('2015-16/*.csv'):
    g1 = pd.read_csv('{}'.format(csv))
    if g1['data_set'].unique()[0] != '2015-2016 Regular Season':
        continue
    else:
        try:
            games.append(game(g1))

        except:
            print csv
            
players = defaultdict(list)
for game in games:
    pscores = collectPlayers(game)
    for i in pscores.keys():
        players[i].append({'fts':pscores[i]['fts'],'shot_distance':pscores[i]['shot_distance'],\
                           'points': pscores[i]['points'], 'shots': pscores[i]['shots']})
        
scoringQualified = {k: v for k, v in players.iteritems() if len(v) >=70}

dill.dump(scoringQualified, open('ArbitraryScoringApp/scoringChamps','wb'))

In [78]:
shot_dist = []

for csv in glob.glob('2015-16/*.csv'):
    g1 = pd.read_csv('{}'.format(csv))
    if g1['data_set'].unique()[0] != '2015-2016 Regular Season':
        continue
    else:
        try:
            shot_dist.append(summaryStats(g1))

        except:
            print csv

In [82]:
shots = reduce(lambda x,y: (x[0]+y[0],x[1]+y[1]), shot_dist)

In [132]:
print [ [i, 1.* shots[0].get(i,0)/shots[1].get(i,1)] for i in range(99)][:51]

[[0, 0.8437591992934943], [1, 0.6227063044592517], [2, 0.5439440106646353], [3, 0.4285026480500722], [4, 0.37752769474239495], [5, 0.3896667323999211], [6, 0.3950091296409008], [7, 0.40369393139841686], [8, 0.39677744209466265], [9, 0.41600227790432803], [10, 0.39260830051499546], [11, 0.403338898163606], [12, 0.4000619770684847], [13, 0.4083076923076923], [14, 0.4118663594470046], [15, 0.4085207100591716], [16, 0.40458309262468706], [17, 0.40657894736842104], [18, 0.39879062736205595], [19, 0.4022117860930162], [20, 0.3993319725366487], [21, 0.38645635028555886], [22, 0.38565368299267255], [23, 0.383383253027738], [24, 0.36440859447498036], [25, 0.3603967304625199], [26, 0.34375875268415645], [27, 0.3332474890548545], [28, 0.32034294621979736], [29, 0.2737306843267108], [30, 0.25252525252525254], [31, 0.19791666666666666], [32, 0.125], [33, 0.18181818181818182], [34, 0.08823529411764706], [35, 0.11538461538461539], [36, 0.06060606060606061], [37, 0.037037037037037035], [38, 0.14705882

In [131]:
print [ [i, shots[1].get(i,0)] for i in range(51)]

[[0, 13588], [1, 19510], [2, 21004], [3, 10385], [4, 5687], [5, 5071], [6, 4929], [7, 4548], [8, 3972], [9, 3512], [10, 3301], [11, 2995], [12, 3227], [13, 3250], [14, 3472], [15, 4225], [16, 5193], [17, 6080], [18, 6615], [19, 6601], [20, 5389], [21, 3677], [22, 2593], [23, 7679], [24, 14009], [25, 20064], [26, 10711], [27, 3883], [28, 1283], [29, 453], [30, 198], [31, 96], [32, 48], [33, 44], [34, 34], [35, 26], [36, 33], [37, 27], [38, 34], [39, 24], [40, 45], [41, 41], [42, 18], [43, 21], [44, 20], [45, 20], [46, 19], [47, 23], [48, 23], [49, 30], [50, 28]]


In [89]:
print [ [i, 2*np.exp(-i/23.)+np.log(1+ i/23.)] for i in range(50)]

[[0, 2.0], [1, 1.9574663505555576], [2, 1.916815512990024], [3, 1.8780297886566968], [4, 1.8410825903369752], [5, 1.8059404108967601], [6, 1.7725644092416162], [7, 1.7409116933766589], [8, 1.7109363619516094], [9, 1.6825903519340719], [10, 1.6558241297051526], [11, 1.6305872549997542], [12, 1.6068288410723199], [13, 1.5844979297929294], [14, 1.5635437967313521], [15, 1.5439161984207241], [16, 1.5255655717250858], [17, 1.5084431934294549], [18, 1.4925013067246666], [19, 1.4776932200938044], [20, 1.4639733831630579], [21, 1.4512974433113499], [22, 1.4396222862044512], [23, 1.4289060629028301], [24, 1.4191082057663316], [25, 1.4101894350257891], [26, 1.4021117575981761], [27, 1.3948384594770291], [28, 1.3883340928248926], [29, 1.3825644587224244], [30, 1.3774965863838653], [31, 1.3730987095262095], [32, 1.3693402404758257], [33, 1.3661917425084151], [34, 1.3636249008434638], [35, 1.3616124926507198], [36, 1.3601283563718911], [37, 1.3591473606143529], [38, 1.358645372833905], [39, 1.35859